#  Hyperparameter verstehen

In diesem Notebook geht es um **Hyperparameter** in Machine Learning und Deep Learning.

Ziele:
- verstehen, was ein *Hyperparameter* ist,
- typische Hyperparameter in neuronalen Netzen kennenlernen,
- besonders: die **Learning Rate** als Beispiel für ein Optimierungsproblem,
- den Bezug zu unserer **Gebäudesegmentierung mit dem U-Net** herstellen.

## 1. Was sind Hyperparameter?

In einem neuronalen Netz gibt es zwei Arten von Größen:

**1. Parameter (Gewichte des Modells)**
- werden **vom Training gelernt** (z. B. Filtergewichte in Convolution-Layern).
- Beispiel U-Net: die Gewichte in den `Conv2d`-Schichten, die lernen, wie Gebäude aussehen.

**2. Hyperparameter**
- werden **nicht** direkt gelernt,
- sondern **von uns eingestellt**, bevor das Training startet.

Typische Hyperparameter:
- Learning Rate (Lernrate)
- Anzahl Epochen (Wie oft sollen alle Bilder durch das Modell geschleust werden während des Trainings?)
- Batch Size (Wie viele Bilder werden gleichzeitig durch das Netz "geschleust")

## 2. Learning Rate – ein Optimierungsproblem

Die **Learning Rate** bestimmt, wie große Schritte das Modell beim Lernen macht.

Wir können uns das wie das Herunterlaufen eines Hügels vorstellen:

- Du stehst irgendwo auf einem Hügel.
- Du möchtest ins **Tal** (Minimum der Fehlerfunktion).
- In jeder Runde schaust du, in welche Richtung es **bergab** geht (Gradient).
- Mit der Learning Rate entscheidest du, **wie große Schritte** du machst.

Was kann passieren?
- Ist die Learning Rate **zu klein**, brauchst du sehr lange, um anzukommen.
- Ist sie **zu groß**, springst du vielleicht am Tal vorbei oder sogar hin und her.
- Es gibt einen guten Bereich → das Training wird schnell und stabil.

Genau das passiert auch beim Training unseres U-Nets für Gebäudesegmentierung: Die Learning Rate bestimmt, wie stark sich die Gewichte nach jedem Batch verändern.

### 2.1 Ein einfaches 2D-Optimierungsproblem

Wir betrachten eine einfache Funktion in 2D, die wie eine **Schüssel** aussieht: Das Minimum dieser Funktion liegt bei 0 - im Zentrum der Schüssel. Wir starten an einem zufälligen Punkt und benutzen **Gradientenabstieg**, um ins Tal zu kommen. Die Learning Rate steuert die Schrittgröße.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

def f(w1, w2):
    return w1**2 + 0.5 * w2**2

def grad_f(w1, w2):
    df_dw1 = 2 * w1
    df_dw2 = 1.0 * w2
    return np.array([df_dw1, df_dw2])

def gradient_descent(start, lr=0.1, n_steps=20):
    w = np.array(start, dtype=float)
    trajectory = [w.copy()]
    for i in range(n_steps):
        g = grad_f(w[0], w[1])
        w = w - lr * g
        trajectory.append(w.copy())
    return np.array(trajectory)

start_point = (2.5, 2.0)
traj_small = gradient_descent(start_point, lr=0.02, n_steps=40)
traj_good  = gradient_descent(start_point, lr=0.1,  n_steps=20)
traj_big   = gradient_descent(start_point, lr=0.4,  n_steps=20)


### 2.2 Vergleich: kleine, gute und große Learning Rate (statisch)

Wir zeichnen die Funktion als Konturplot und darüber die Wege, die Gradient Descent für verschiedene Learning Rates geht.

In [ ]:
start_point = (2.5, 2.0)

# bewusst gewählte Lernraten
lr_small = 0.02     # zu klein
lr_good  = 0.15      # gut
lr_big   = 0.95      # zu groß (über Stabilitätsgrenze)

traj_small = gradient_descent(start_point, lr=lr_small, n_steps=40)
traj_good  = gradient_descent(start_point, lr=lr_good,  n_steps=20)
traj_big   = gradient_descent(start_point, lr=lr_big,   n_steps=20)

In [ ]:
plt.figure(figsize=(6, 6))
cs = plt.contour(W1, W2, Z, levels=20)
plt.clabel(cs, inline=True, fontsize=8)

plt.plot(traj_small[:,0], traj_small[:,1], 'o-', label=f'LR = {lr_small} (zu klein)')
plt.plot(traj_good[:,0],  traj_good[:,1],  'o-', label=f'LR = {lr_good} (gut)')
plt.plot(traj_big[:,0],   traj_big[:,1],   'o-', label=f'LR = {lr_big} (zu groß)')

plt.scatter([0],[0], color='red', marker='x', s=100, label='Minimum')
plt.xlabel('w1')
plt.ylabel('w2')
plt.title('Gradientenabstieg für verschiedene Learning Rates')
plt.legend()
plt.grid(True)
plt.show()


### 2.3 2D-Animation für eine Learning Rate

Jetzt animieren wir den Weg eines Punktes für bestimmte Learning Rates (z. B. `lr = 0.02`, `lr = 0.15`, `lr = 0.95`). Das entspricht der **Optimierung der Gewichte** in unserem U-Net – nur dass wir hier statt Millionen Gewichten
nur zwei Parameter haben und sie sehen können.

In [ ]:
lrs    = [lr_small, lr_good, lr_big]
labels = [
    f"LR = {lr_small} (zu klein)",
    f"LR = {lr_good} (gut)",
    f"LR = {lr_big} (zu groß)"
]
trajs  = [traj_small, traj_good, traj_big]
colors = ["blue", "red", "green"]

max_steps = max(len(t) for t in trajs)

fig, ax = plt.subplots(figsize=(6, 6))

cs = ax.contour(W1, W2, Z, levels=20)
ax.clabel(cs, inline=True, fontsize=8)

points = []
paths = []
for c, lab in zip(colors, labels):
    p, = ax.plot([], [], "o", color=c, label=lab)
    line, = ax.plot([], [], "--", color=c, alpha=0.7)
    points.append(p)
    paths.append(line)

ax.scatter([0], [0], color="black", marker="x", s=80, label="Minimum")
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_xlabel("w1")
ax.set_ylabel("w2")
ax.set_title("Gradientenabstieg für verschiedene Learning Rates")
ax.legend(loc="upper right")
ax.grid(True)

def init():
    for p, line in zip(points, paths):
        p.set_data([], [])
        line.set_data([], [])
    return points + paths

def update(frame):
    for traj, p, line in zip(trajs, points, paths):
        idx = min(frame, len(traj) - 1)
        w = traj[idx]
        p.set_data([w[0]], [w[1]])                      # Punkt als Sequenz
        line.set_data(traj[:idx+1, 0], traj[:idx+1, 1]) # Pfad
    ax.set_title(f"Gradientenabstieg – Schritt {frame}")
    return points + paths

anim = animation.FuncAnimation(
    fig,
    update,
    init_func=init,
    frames=max_steps,
    interval=500,
    blit=True
)

plt.close(fig)
HTML(anim.to_jshtml())


## 3. Bezug zur Gebäudesegmentierung

In unserem U-Net für Gebäudesegmentierung gibt es **sehr viele Parameter** (Gewichte in allen Convolution-Layern).
Die Optimierung dieser Gewichte passiert mit genau derselben Idee wie im 2D-Beispiel:

- Wir berechnen einen **Fehler** (Loss) zwischen Vorhersage-Maske (Prediction) und Ground Truth.
- Wir berechnen den **Gradienten** dieses Fehlers bezüglich aller Gewichte.
- Wir machen einen Schritt in Richtung "bergab".
- Die **Learning Rate** bestimmt die Schrittgröße.

Bei zu großer Learning Rate könnte das U-Net instabil lernen, bei zu kleiner Learning Rate lernt es nur sehr langsam.
In der Praxis testet man oft mehrere Learning Rates und wählt einen Bereich, in dem das Training stabil und schnell ist.

## 4. Weitere wichtige Hyperparameter

### 4.1 Anzahl Epochen
- Eine **Epoche** bedeutet: das Netz hat alle Trainingsbeispiele einmal gesehen.
- Wenige Epochen → das Netz hat zu wenig gelernt (Underfitting).
- Zu viele Epochen → Gefahr von **Overfitting**.

### 4.2 Batch-Größe
- Statt **ein Bild nach dem anderen** zu verarbeiten, nehmen wir Batches (z. B. 8 Bilder).
- Kleine Batches:
  - mehr Rauschen im Gradienten,
  - oft besser für Generalisierung, aber langsamer.
- Große Batches:
  - glattere Gradienten,
  - effizienter auf GPUs, aber brauchen mehr Speicher.

### 4.3 Modelltiefe und Anzahl der Filter
- Mehr Encoder-/Decoder-Stufen → das U-Net sieht mehr **Kontext**.
- Mehr Filter → das Netz kann mehr Muster lernen.
- Aber: mehr Parameter = mehr Rechenzeit + mehr Overfitting-Risiko.
In unserem Mini-U-Net haben wir bewusst ein **kleines Modell** gewählt, damit es im Workshop schnell trainierbar bleibt.

## 5. Mini-Übungsaufgaben

1. **Startpunkt variieren:**
   - Ändert `start_point = (2.5, 2.0)` zu z. B. `(-2, 2)` oder `(1, -3)`.
   - Was passiert mit der Trajektorie?

2. **Verbindung zum U-Net:**
   - Öffnet das U-Net-Notebook.
   - Sucht die Stelle mit `optimizer = torch.optim.Adam(..., lr=1e-3)`.
   - Überlegt: Was würde passieren, wenn ihr `1e-2` oder `1e-4` nehmt?